Ev Fiyatları verilerini Kaggle'dan indirin.

Verilerinizi eğitim ve test kümelerine ayırın.

Modelinizi eğitim kümesinden önceki derste olduğu gibi oluşturun. Modelinizin uygunluğunu değerlendirin.

Test kümesindeki ev fiyatlarını tahmin edin ve bu derste belirttiğimiz metrikleri kullanarak modelinizin performansını değerlendirin.

Modelinizin performansı tatmin edici mi? Açıklayın.

Bazı değişkenleri ekleyerek veya çıkararak modelinizi tahmin performansı açısından geliştirmeye çalışın

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse
from sklearn.preprocessing import scale

%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
pd.options.display.max_columns = None

house = pd.read_csv("train.csv")
house.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.000,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.000,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.000,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.000,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.000,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.000,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.000,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.000,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.000,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.000,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
#önce kategorik değişkenleri dönüştürerek baslayalım.
KitchenQual = {"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["KitchenQual"] = house["KitchenQual"].map(KitchenQual)

ExterQual = {"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["ExterQual"] = house["ExterQual"].map(ExterQual)

ExterCond = {"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["ExterCond"] = house["ExterCond"].map(ExterCond)

HeatingQC = {"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["HeatingQC"] = house["HeatingQC"].map(HeatingQC)

FireplaceQu = {"NA":0,"Po":1,"Fa":2,"TA":3,"Gd":4,"Ex":5}
house["FireplaceQu"] = house["FireplaceQu"].map(FireplaceQu)

In [4]:
#kategorik değişkenleri get_dummies ile dönüştürelim
get_dummies_house = pd.get_dummies(house,drop_first=True)


In [5]:
# sonra da değişken değerlerini standartlaştıralım.

house_names=get_dummies_house.select_dtypes(include=["float","integer"]).columns
house_scaled_df = scale(get_dummies_house[house_names])
house_scaled_df = pd.DataFrame(house_scaled_df,columns=house_names)

In [6]:
#SlePrice değişkeni ile en cok korelasyonu olan ilk 30 değişkene bakalım.
house_corr=house_scaled_df.corr().SalePrice.abs().sort_values(ascending=False).index[0:30]
house_scaled_df[house_corr].corr().SalePrice.head()

SalePrice     1.000
OverallQual   0.791
GrLivArea     0.709
ExterQual     0.683
KitchenQual   0.660
Name: SalePrice, dtype: float64

In [7]:
#SalePrice hedef değişkeni ile en yüksek collinearitye sahip olup aralarında multicollinearity 
#düşük olan değişkenler üzerinden model kuralım.

house_scaled_df[['OverallQual', 'GrLivArea', 'KitchenQual',
       'GarageCars', 'YearBuilt',  'Foundation_PConc',
        'Fireplaces','BsmtFinType1_GLQ', 'HeatingQC','Neighborhood_NridgHt',"TotalBsmtSF", 'BsmtFinSF1', 'SaleType_New',
       'GarageType_Detchd', 'SaleCondition_Partial', 'LotFrontage',
       'Foundation_CBlock', 'GarageType_Attchd']].corr().head()

,OverallQual,GrLivArea,KitchenQual,GarageCars,YearBuilt,Foundation_PConc,Fireplaces,BsmtFinType1_GLQ,HeatingQC,Neighborhood_NridgHt,TotalBsmtSF,BsmtFinSF1,SaleType_New,GarageType_Detchd,SaleCondition_Partial,LotFrontage,Foundation_CBlock,GarageType_Attchd
OverallQual,1.000,0.593,0.673,0.601,0.572,0.568,0.397,0.433,0.457,0.369,0.538,0.240,0.327,-0.337,0.323,0.252,-0.430,0.358
GrLivArea,0.593,1.000,0.421,0.467,0.199,0.293,0.462,0.194,0.255,0.180,0.455,0.208,0.168,-0.232,0.163,0.403,-0.267,0.140
KitchenQual,0.673,0.421,1.000,0.510,0.530,0.538,0.256,0.420,0.504,0.354,0.433,0.228,0.338,-0.269,0.340,0.182,-0.401,0.273
GarageCars,0.601,0.467,0.510,1.000,0.538,0.453,0.301,0.321,0.325,0.283,0.435,0.224,0.286,-0.153,0.282,0.286,-0.319,0.321
YearBuilt,0.572,0.199,0.530,0.538,1.000,0.651,0.148,0.479,0.449,0.269,0.391,0.250,0.347,-0.487,0.344,0.123,-0.291,0.472


### 1.OLS

In [10]:
Y = house_scaled_df["SalePrice"]
X = house_scaled_df[['OverallQual', 'GrLivArea', 'KitchenQual',
       'GarageCars', 'YearBuilt', 'Fireplaces','BsmtFinType1_GLQ', "TotalBsmtSF",'HeatingQC','Neighborhood_NridgHt', 'BsmtFinSF1', 'SaleType_New',
       'GarageType_Detchd']]

X_egitim,X_test,y_egitim,y_test = train_test_split(X,Y,test_size=0.2,random_state=465)

lrm = LinearRegression()
lrm.fit(X_egitim, y_egitim)

y_egitim_tahmini = lrm.predict(X_egitim)
y_test_tahmini = lrm.predict(X_test)

print("Eğitim kümesindeki gözlem sayısı  : {}".format(X_egitim.shape[0]))
print("Test kümesindeki gözlem sayısı    : {}".format(X_test.shape[0]), "\n")

print("Eğitim kümesindeki R-Kare değeri  : {}".format(lrm.score(X_egitim, y_egitim)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(lrm.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesindeki gözlem sayısı  : 1168
Test kümesindeki gözlem sayısı    : 292 

Eğitim kümesindeki R-Kare değeri  : 0.7996167871087067
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : 0.8188369873932205
Ortalama Mutlak Hata (MAE)        : 0.2950452009685232
Ortalama Kare Hata (MSE)          : 0.2081237476814523
Kök Ortalama Kare Hata (RMSE)     : 0.4562058172376283
Ortalama Mutlak Yüzde Hata (MAPE) : 152.83798885997945


### 2.Ridge

In [19]:
Y = house_scaled_df["SalePrice"]
X = house_scaled_df[['OverallQual', 'GrLivArea', 'KitchenQual',
       'GarageCars', 'YearBuilt', 'Fireplaces','BsmtFinType1_GLQ', "TotalBsmtSF",'HeatingQC','Neighborhood_NridgHt', 'BsmtFinSF1', 'SaleType_New',
       'GarageType_Detchd']]

X_egitim,X_test,y_egitim,y_test = train_test_split(X,Y,test_size=0.2,random_state=465)

from sklearn.linear_model import Ridge


ridgereg = Ridge(alpha=10**20)
ridgereg.fit(X_egitim, y_egitim)

y_egitim_tahmini = ridgereg.predict(X_egitim)
y_test_tahmini = ridgereg.predict(X_test)

print("Eğitim kümesindeki gözlem sayısı  : {}".format(X_egitim.shape[0]))
print("Test kümesindeki gözlem sayısı    : {}".format(X_test.shape[0]), "\n")

print("Eğitim kümesindeki R-Kare değeri  : {}".format(ridgereg.score(X_egitim, y_egitim)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(ridgereg.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesindeki gözlem sayısı  : 1168
Test kümesindeki gözlem sayısı    : 292 

Eğitim kümesindeki R-Kare değeri  : 0.0
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : -0.0019389852335489621
Ortalama Mutlak Hata (MAE)        : 0.7422682414480514
Ortalama Kare Hata (MSE)          : 1.1510478521770509
Kök Ortalama Kare Hata (RMSE)     : 1.0728689818319155
Ortalama Mutlak Yüzde Hata (MAPE) : 108.22377601311794


### 3.Lasso

In [25]:
Y = house_scaled_df["SalePrice"]
X = house_scaled_df[['OverallQual', 'GrLivArea', 'KitchenQual',
       'GarageCars', 'YearBuilt', 'Fireplaces','BsmtFinType1_GLQ', "TotalBsmtSF",'HeatingQC','Neighborhood_NridgHt', 'BsmtFinSF1', 'SaleType_New',
       'GarageType_Detchd']]

X_egitim,X_test,y_egitim,y_test = train_test_split(X,Y,test_size=0.2,random_state=465)

from sklearn.linear_model import Lasso


lassoreg = Lasso(alpha=10**20.5)
lassoreg.fit(X_egitim, y_egitim)

y_egitim_tahmini = lassoreg.predict(X_egitim)
y_test_tahmini = lassoreg.predict(X_test)

print("Eğitim kümesindeki gözlem sayısı  : {}".format(X_egitim.shape[0]))
print("Test kümesindeki gözlem sayısı    : {}".format(X_test.shape[0]), "\n")

print("Eğitim kümesindeki R-Kare değeri  : {}".format(lassoreg.score(X_egitim, y_egitim)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(lassoreg.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesindeki gözlem sayısı  : 1168
Test kümesindeki gözlem sayısı    : 292 

Eğitim kümesindeki R-Kare değeri  : 0.0
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : -0.0019389852335489621
Ortalama Mutlak Hata (MAE)        : 0.7422682414480514
Ortalama Kare Hata (MSE)          : 1.1510478521770509
Kök Ortalama Kare Hata (RMSE)     : 1.0728689818319155
Ortalama Mutlak Yüzde Hata (MAPE) : 108.22377601311794


### 4.ElasticNet

In [26]:
Y = house_scaled_df["SalePrice"]
X = house_scaled_df[['OverallQual', 'GrLivArea', 'KitchenQual',
       'GarageCars', 'YearBuilt', 'Fireplaces','BsmtFinType1_GLQ', "TotalBsmtSF",'HeatingQC','Neighborhood_NridgHt', 'BsmtFinSF1', 'SaleType_New',
       'GarageType_Detchd']]

X_egitim,X_test,y_egitim,y_test = train_test_split(X,Y,test_size=0.2,random_state=465)

from sklearn.linear_model import ElasticNet


elasticreg = ElasticNet(alpha=10**15,l1_ratio=0.5)
elasticreg.fit(X_egitim, y_egitim)

y_egitim_tahmini = elasticreg.predict(X_egitim)
y_test_tahmini = elasticreg.predict(X_test)

print("Eğitim kümesindeki gözlem sayısı  : {}".format(X_egitim.shape[0]))
print("Test kümesindeki gözlem sayısı    : {}".format(X_test.shape[0]), "\n")

print("Eğitim kümesindeki R-Kare değeri  : {}".format(elasticreg.score(X_egitim, y_egitim)))
print("-----Test kümesi istatistikleri---")
print("Test kümesi R-Kare değeri         : {}".format(elasticreg.score(X_test, y_test)))
print("Ortalama Mutlak Hata (MAE)        : {}".format(mean_absolute_error(y_test, y_test_tahmini)))
print("Ortalama Kare Hata (MSE)          : {}".format(mse(y_test, y_test_tahmini)))
print("Kök Ortalama Kare Hata (RMSE)     : {}".format(rmse(y_test, y_test_tahmini)))
print("Ortalama Mutlak Yüzde Hata (MAPE) : {}".format(np.mean(np.abs((y_test - y_test_tahmini) / y_test)) * 100))

Eğitim kümesindeki gözlem sayısı  : 1168
Test kümesindeki gözlem sayısı    : 292 

Eğitim kümesindeki R-Kare değeri  : 0.0
-----Test kümesi istatistikleri---
Test kümesi R-Kare değeri         : -0.0019389852335489621
Ortalama Mutlak Hata (MAE)        : 0.7422682414480514
Ortalama Kare Hata (MSE)          : 1.1510478521770509
Kök Ortalama Kare Hata (RMSE)     : 1.0728689818319155
Ortalama Mutlak Yüzde Hata (MAPE) : 108.22377601311794
